In [ ]:
intention = '''Create an easy/safe API to parse/hold/yield constructions of pydantic docx class objects.
In theory this can be generalized and abstracted since the classes are largely absracted. 
However since this is API based, I will just focus on solutions for THIS dictionary task. These can be expanded later, or just have other APIs added
Final objective is have a tool to pass complex commands/instructions, and have it yield a pandas dataframe of the results.
Intermediate steps may be yield a list of lists
I am not yet sure if there is benefit trying to have the whole dictionary in one notional object. 
I don't think so, since each operation is psuedo atomic, and any aggregate actions are intended to be done in something like pandas.
Eventually I would want a loop for pandas operations to easily feed updates to the data. 
'''
conclusion = '''create a pular dictionary specifc datastructure to handle the aggregate paragraphs. 
This will only be per lemma, 
as any aggregations from roots and relations of that sort can be managed in pandas and created by simple inheretance/reference
'''

In [1]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs #type:ignore
import re

In [ ]:
outputs = ['txt_FulaAnnotations','txt_FulaLemmas','txt_FulaRoots',
'txt_FulaSenseEnglish','txt_FulaSenseFrench',
'txt_FulaSenseClassifications',
'txt_FulaSenseEnglishAnnotations','txt_FulaSenseFrenchAnnotations',
'txt_FulaDialects','txt_FulaPOSTags','txt_FulaSynonyms',
'txt_FulaInParenthesis','txt_POS','txt_RootOrigins']

In [3]:
noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
print(nounPatternRegex)
s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
print(re.findall(nounPatternRegex,s))
print(nounPatternRegex.findall(s))

# if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# so have a nouns list? nouns may then be 
# [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# ([[{word}],{Optional[class]}]),  #singular
   #([[{word}],{Optional[class]}])]   #plural

# so nounStruct[0][1] is the flag for noun?
   # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

{'title': 'Docx_Paragraph_and_Runs', 'description': "input:   paragraph = your_paragraph_here\n\nwhen given a docx document's paragraph object, will parse it to a specified schema", 'type': 'object', 'properties': {}}


In [ ]:
def getAssert(dct,val,typ):
   out = dct.get(val,False)

In [ ]:
class dict_entry(pydantic.BaseModel):
   irregularities: List[str]
   paragraphs: List[Docx_Paragraph_and_Runs]
   lemma: str
   root: List[Tuple[int,str,str]] #index,root_text, root_meta_data
   englishGloss: str
   frenchGloss: str

   class Config:
      validate_all = True
      # extra = 'forbid'
      validate_assignment = True
      smart_union = True  

   @root_validator(pre=True)
   def _validate_and_build(cls, values: Dict[str,Any]) -> Dict[str, Any]:


   def give_entryText(self, joiner = '\t') -> str:
      return joiner.join([para.trustyGet('para_text') for para in self.paragraphs])

   def give_root_text(self, main = False, sub = False, both = True) -> str:
      if both:
         return ''.join([r[1] for r in self.root])
      if main:
         return  str(self.root[0][1])
      if sub:
         return str(self.root[1][1])
      raise RuntimeError('did not give valid input')

   def trustyGet(self, feat: str, silent_return = True) -> str:
      if not silent_return:
         output : str = getattr(self,feat,'')
         if output:
            return output
         else:
            raise AttributeError(f'trusty getter could not find: {feat}')
      else:
         output : str = getattr(self,feat,'')
         return output
      
def getAssert(dct,val,typ):
   out = dct.get(val,False)
            
